In [0]:
#%run ./includes/configuration

Out[61]: DataFrame[]

In [0]:
#%run ./includes/utilities

In [0]:
import json

movies = {"movie": []}
for i in range(8):
    with open(myPath + f"movie_{i}.json") as f:
        data = json.load(f)
        movies["movie"].append(data["movie"])

dbutils.fs.put(rawPath, json.dumps(movies, indent=2), True)

In [0]:
#Display the files in the raw path
display(dbutils.fs.ls(f"FileStore/tables/movie_0.json"))

path,name,size,modificationTime
dbfs:/FileStore/tables/movie_0.json,movie_0.json,1907852,1661141670000


In [0]:
#Make Notebook Idempotent
dbutils.fs.rm(bronzePath, recurse=True)

Out[69]: False

In [0]:
#Ingest raw data
rawDF = ingest_batch_raw(rawData).cache()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1430919566441882> in <module> 
 1 #Ingest raw data 
 ----> 2 rawDF = ingest_batch_raw ( rawData ) . cache ( ) 

 NameError : name 'ingest_batch_raw' is not defined

In [0]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import *

raw_movie_DF = rawDF.select(explode("movie").alias("movie"))
display(raw_movie_DF)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1809201671029017> in <module> 
 2 from pyspark . sql . functions import * 
 3 
 ----> 4 raw_movie_DF = rawDF . select ( explode ( "movie" ) . alias ( "movie" ) ) 
 5 display ( raw_movie_DF ) 

 NameError : name 'rawDF' is not defined

In [0]:
# Ingestion Metadata
from pyspark.sql.functions import current_timestamp, lit
raw_movie_data_df = (raw_movie_DF
                     .select("movie",
                             lit("files.training.databricks.com").alias("datasource"),
                             current_timestamp().alias("ingesttime"),
                             lit("new").alias("status"),
                             current_timestamp().cast("date").alias("ingestdate")
                            )
                    )

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3241425325867594> in <module> 
 1 # Ingestion Metadata 
 2 from pyspark . sql . functions import current_timestamp , lit
 ----> 3 raw_movie_data_df = (raw_movie_DF
 4 .select("movie",
 5 lit ( "files.training.databricks.com" ) . alias ( "datasource" ) , 

 NameError : name 'raw_movie_DF' is not defined

In [0]:
#Partitioning
from pyspark.sql.functions import col
(raw_movie_data_df.select("datasource",
                          "ingesttime",
                          "value",
                          "status",
                          col("ingestdate").alias("p_ingestdate"))
 .write.format("delta")
 .mode("append")
 .partitionBy("p_ingestdate")
 .save(bronzePath)
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3241425325867606> in <module> 
 1 #Partitioning 
 2 from pyspark . sql . functions import col
 ----> 3 (raw_movie_data_df.select("datasource",
 4 "ingesttime" , 
 5 "value" , 

 NameError : name 'raw_movie_data_df' is not defined

In [0]:
display(dbutils.fs.ls(bronzePath))

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3241425325867608> in <module> 
 ----> 1 display ( dbutils . fs . ls ( bronzePath ) ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: File /6793640940748933/FileStore/tables/movie_0.json/bronze does not exist.
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.listStatus(NativeAzureFileSystem.java:2492)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:95)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:66)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:444)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:33)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:31)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:204)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:240)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:225)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:419)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:339)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperationWithResultTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:302)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:510)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:164)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$ls$1(DBUtilsCore.scala:157)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:91)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:155)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invo

In [0]:
spark.sql("""
drop table if exists movie_bronze
""")

spark.sql(f"""
create table movie_bronze
using delta
location "{bronzePath}"
""")